In [1]:
import pandas as pd
import numpy as np
import os
import math

In [2]:
def apply_pbc(value):
    if abs(value) > 0.5:
        return 1 - abs(value)
    return value


def mic_eucledian_distance(coor1, coor2):
    x_coor1, y_coor1, z_coor1 = coor1
    x_coor2, y_coor2, z_coor2 = coor2
    
    delta_x = x_coor1 - x_coor2
    delta_y = y_coor1 - y_coor2
    delta_z = z_coor1 - z_coor2

    distance = math.sqrt(sum([(apply_pbc(delta_x))**2, (apply_pbc(delta_y))**2, (apply_pbc(delta_z))**2]))
    return distance


def get_duplicate_coor_final_closest_w_data(dict):
    duplicate_coor_final_closest = {}
    for i, values in dict.items():
        for entry in values:
            coor_init = entry["coor_init"]
            coor_final_closest = entry["coor_final_closest"]
            dist = entry["dist"]

        if coor_final_closest in duplicate_coor_final_closest:
            duplicate_coor_final_closest[coor_final_closest].append({"coor_init": coor_init, "dist": dist})
        else:
            duplicate_coor_final_closest[coor_final_closest] = [{"coor_init": coor_init, "dist": dist}]

    duplicate_coor_final_closest_w_data = {}
    for coor_final_closest, coor_inits_dists in duplicate_coor_final_closest.items():
        if len(coor_inits_dists) > 1:
            duplicate_coor_final_closest_w_data[coor_final_closest] = [{"coor_inits and dists": coor_inits_dists}]

    return duplicate_coor_final_closest_w_data

# def get_coordinate_POSCAR(file_path):
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
#     data = lines[poscar_line_nr_start:poscar_line_nr_start+amount_Li]

#     df = pd.DataFrame([string.strip().split() for string in data])
#     df_float = df.astype(float)

#     check_negative = df_float.lt(0).any().any()

#     if check_negative == True:
#         breaksys.exit()    

#     coordinate = [row.to_numpy() for index, row in df_float.iterrows()]

#     return coordinate

In [3]:
# class PoscarReader:
#     def __init__(self, file_path):
#         self.file_path = file_path
#         self.poscar_line_nr_start = 8   # index from 0
#         self.poscar_line_nr_end = 60
#         self.amount_Li = 24
#         self.n_decimal = 16
        

#     def get_coordinate(self):
#         with open(self.file_path, 'r') as file:
#             lines = file.readlines()

#         data = lines[self.poscar_line_nr_start : self.poscar_line_nr_start+self.amount_Li]
#         data_non_Li = lines[self.poscar_line_nr_start+self.amount_Li : self.poscar_line_nr_end]

#         df = pd.DataFrame([string.strip().split() for string in data])
#         df_float = df.astype(float)

#         df_non_Li = pd.DataFrame([string.strip().split() for string in data_non_Li])
#         df_float_non_Li = df_non_Li.astype(float)

#         check_negative = df_float.lt(0).any().any()

#         if check_negative:
#             sys.exit()

#         coordinates = [row.to_numpy() for index, row in df_float.iterrows()]

#         return coordinates

In [4]:
file_path_init = "7_2_CONTCAR_init"
file_path = "1_2_CONTCAR"

In [5]:
# poscar_reader_init = PoscarReader(file_path_init)
# coordinate_init = poscar_reader_init.get_coordinate()

# poscar_reader = PoscarReader(file_path)
# coordinate = poscar_reader.get_coordinate()

In [6]:
poscar_line_nr_start = 8        # index from 0
poscar_line_nr_end = 60
amount_Li = 24
n_decimal = 16

In [7]:
with open(file_path_init, 'r') as file_init:
    lines_init = file_init.readlines()
data_init = lines_init[poscar_line_nr_start:poscar_line_nr_start+amount_Li]

df_init = pd.DataFrame([string.strip().split() for string in data_init])
df_float_init = df_init.astype(float)

check_negative_init = df_float_init.lt(0).any().any()

if check_negative_init == True:
    breaksys.exit()    

coordinate_init = [row.to_numpy() for index, row in df_float_init.iterrows()]

In [8]:
with open(file_path, 'r') as file:
    lines = file.readlines()
data = lines[poscar_line_nr_start:poscar_line_nr_start+amount_Li]
data_non_Li = lines[poscar_line_nr_start+amount_Li:poscar_line_nr_end]

df = pd.DataFrame([string.strip().split() for string in data])
df_float = df.astype(float)
df_non_Li = pd.DataFrame([string.strip().split() for string in data_non_Li])
df_float_non_Li = df_non_Li.astype(float)

check_negative = df_float.lt(0).any().any()

if check_negative == True:
    breaksys.exit()

coordinate = [row.to_numpy() for index, row in df_float.iterrows()]

In [9]:
coor_final_reindex_dict_value = {}
coor_final_reindex_dict = {}

for i, coor_init in enumerate(coordinate_init):
    coor_final_reindex_dict_value['coor_init'] = tuple(coor_init)
    coor_final_reindex_dict_value['coor_final_closest'] = None
    coor_final_reindex_dict_value['dist'] = None

    coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]


def get_coor_final_reindex_dict(coor_final_reindex_dict, coordinate):
    # coor_final_reindex = []
    # coor_final_reindex_dict = {}

    for i, value in coor_final_reindex_dict.items():
        distance_prev = float("inf")
        coor_final_closest = None

        for entry in value:
            coor_init = entry['coor_init']

            for j, coor in enumerate(coordinate):
                distance = mic_eucledian_distance(coor_init, coor)

                if distance < distance_prev:
                    distance_prev = distance
                    coor_final_closest = coor

        # coor_final_reindex.append(coor_final_closest)

            if entry['coor_final_closest'] == None and entry['dist'] == None:
                entry['coor_final_closest'] = tuple(coor_final_closest)
                entry['dist'] = distance_prev

        # if i in coor_final_reindex_dict:
        # new_entry = coor_final_reindex_dict[i].copy()
        # new_entry.append(coor_final_reindex_dict_value)
        # coor_final_reindex_dict[i] = new_entry
        # else:
        #     coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

    return coor_final_reindex_dict


def get_coor_final_reindex_dict_eliminated(coor_final_reindex_dict, duplicate_coor_final_w_data):
    for key, value in duplicate_coor_final_w_data.items():
        # Extract the list of coor_inits and dists
        coor_inits_and_dists = value[0]['coor_inits and dists']
        
        # # Find the coor_init with the biggest distance, and then delete it from coordinate
        max_distance_coor_init = max(coor_inits_and_dists, key=lambda x: x['dist'])['coor_init']

        # print("Duplicate coor_final_closest:", key)
        # # for i in coor_inits_and_dists:
        # #     print(i['coor_init'])
        # print("coor_init with the biggest distance:", max_distance_coor_init)
        # # print()

        coor_final_reindex_dict_eliminated = coor_final_reindex_dict.copy()

        # Find the corresponding entry and delete it
        for idx, values in coor_final_reindex_dict_eliminated.items():
            for entry in values:
                if entry['coor_init'] == max_distance_coor_init and entry['coor_final_closest'] == key:
                    entry['coor_final_closest'] = None
                    entry['dist'] = None
                    # del entry[0]['coor_final_closest']
                    # del entry[0]['dist']
                    # # entry['coor_init'] = max_distance_coor_init

    return coor_final_reindex_dict_eliminated


def get_coor_final_closest_eliminated(coor_final_reindex_dict_eliminated, coordinate):
    # coor_init_array = []
    coor_final_closest_eliminated_array = coordinate.copy()

    for key, values in coor_final_reindex_dict_eliminated.items():
        for entry in values:
            if np.all(entry['coor_final_closest'] == coordinate):
                coor_final_closest_eliminated_array.remove(entry['coor_final_closest'])

    return coor_final_closest_eliminated_array


    # # coor_init_array = np.array(coor_init_array)
    # coor_final_closest_eliminated_array = np.array(coor_final_closest_eliminated_array)


    # idx_empty = []

    # for idx, values in coor_final_reindex_dict_eliminated.items():
    #     for entry in values:
    #         if entry['dist'] == None:
    #             idx_empty.append(idx)
    #             print(idx)

In [10]:
# # coor_final_reindex_dict, duplicate_coor_f    
# coor_final_reindex_dict_value = {}
# coor_final_reindex_dict = {}

# for i, coor_init in enumerate(coordinate_init):
#     distance_prev = float("inf")
#     closest24 = None

#     for j, coor in enumerate(coordinate):
#         distance = mic_eucledian_distance(coor_init, coor)

#         if distance < distance_prev:
#             distance_prev = distance
#             coor_final_closest = coor

#     # coor_final_reindex.append(coor_final_closest)

#     coor_final_reindex_dict_value['coor_init'] = tuple(coor_init)
#     coor_final_reindex_dict_value['coor_final_closest'] = tuple(coor_final_closest)
#     coor_final_reindex_dict_value['dist'] = distance_prev

#     if i in coor_final_reindex_dict:
#         new_entry = coor_final_reindex_dict[i].copy()
#         new_entry.append(coor_final_reindex_dict_value)
#         coor_final_reindex_dict[i] = new_entry
#     else:
#         coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

#     duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)

In [11]:
# # coor_final_reindex_dict_value = {}
# # coor_final_reindex_dict = {}

# # for i, coor_init in enumerate(coordinate_init):
# #     coor_final_reindex_dict_value['coor_init'] = tuple(coor_init)

# #     coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

In [12]:
# coor_final_reindex_dict

In [13]:
# # # coor_final_reindex = []
# # coor_final_reindex_dict_value = {}
# # # coor_final_reindex_dict = {}

# # for i, value in coor_final_reindex_dict.items():
# #     distance_prev = float("inf")
# #     closest24 = None

# #     for entry in value:
# #         coor_init = entry['coor_init']

# #         for j, coor in enumerate(coordinate):
# #             distance = mic_eucledian_distance(coor_init, coor)

# #             if distance < distance_prev:
# #                 distance_prev = distance
# #                 coor_final_closest = coor

# #     # coor_final_reindex.append(coor_final_closest)

# #         entry['coor_final_closest'] = tuple(coor_final_closest)
# #         entry['dist'] = distance_prev

# #     # if i in coor_final_reindex_dict:
# #     # new_entry = coor_final_reindex_dict[i].copy()
# #     # new_entry.append(coor_final_reindex_dict_value)
# #     # coor_final_reindex_dict[i] = new_entry
# #     # else:
# #     #     coor_final_reindex_dict[i] = [coor_final_reindex_dict_value.copy()]

# # duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)
        
# #     # if tuple(coor_init) in coor_final_reindex_dict:
# #     #     new_entry = coor_final_reindex_dict[tuple(coor_init)].copy()
# #     #     new_entry.append(coor_final_reindex_dict_value)
# #     #     coor_final_reindex_dict[tuple(coor_init)] = new_entry
# #     # else:
# #     #     coor_final_reindex_dict[tuple(coor_init)] = [coor_final_reindex_dict_value.copy()]

In [14]:
# coor_final_reindex_dict

In [15]:
# get init dict
coor_final_reindex_dict = get_coor_final_reindex_dict(coor_final_reindex_dict, coordinate)
# get duplicate
duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)
# works only if duplicate 1 atm
if len(duplicate_coor_final_w_data) > 1:
    sys.exit()
elif len(duplicate_coor_final_w_data) == 1:
    # # get dict eliminated
    # coor_final_reindex_dict_eliminated = get_coor_final_reindex_dict_eliminated(coor_final_reindex_dict, duplicate_coor_final_w_data)
    for key, value in duplicate_coor_final_w_data.items():
        # Extract the list of coor_inits and dists
        coor_inits_and_dists = value[0]['coor_inits and dists']
        
        # # Find the coor_init with the biggest distance, and then delete it from coordinate
        max_distance_coor_init = max(coor_inits_and_dists, key=lambda x: x['dist'])['coor_init']

        # print("Duplicate coor_final_closest:", key)
        # # for i in coor_inits_and_dists:
        # #     print(i['coor_init'])
        # print("coor_init with the biggest distance:", max_distance_coor_init)
        # # print()

        coor_final_reindex_dict_eliminated = coor_final_reindex_dict.copy()

        # Find the corresponding entry and delete it
        for idx, values in coor_final_reindex_dict_eliminated.items():
            for entry in values:
                if entry['coor_init'] == max_distance_coor_init and entry['coor_final_closest'] == key:
                    entry['coor_final_closest'] = None
                    entry['dist'] = None
                    # del entry[0]['coor_final_closest']
                    # del entry[0]['dist']
                    # # entry['coor_init'] = max_distance_coor_init

    coor_final_closest_eliminated_array = coordinate.copy()

    for key, values in coor_final_reindex_dict_eliminated.items():
        for entry in values:
            for i in coordinate:
                if (entry['coor_final_closest'] == i).any():
                    # coor_final_closest_eliminated_array.remove(entry['coor_final_closest'])
                    coor_final_closest_eliminated_array = [arr for arr in coor_final_closest_eliminated_array if not np.array_equal(arr, i)]
    
    # # get coor eliminated
    # coor_final_closest_eliminated_array = get_coor_final_closest_eliminated(coor_final_reindex_dict_eliminated, coordinate)

    # repeat
    coor_final_reindex_dict_eliminated_final = get_coor_final_reindex_dict(coor_final_reindex_dict_eliminated, coor_final_closest_eliminated_array)

In [16]:
coor_final_closest_eliminated_array

[array([0.02087593, 0.69994202, 0.28260712])]

In [17]:
duplicate_coor_final_w_data

{(0.1710569269062762,
  0.9668364475617172,
  0.1690830191903037): [{'coor_inits and dists': [{'coor_init': (0.0731773730080812,
      0.7875552700544789,
      0.0790869644534405),
     'dist': 0.22320716285077413},
    {'coor_init': (0.1995770681557207, 0.9885209462851521, 0.2248956568588323),
     'dist': 0.0663224431493094}]}]}

In [18]:
coor_final_reindex_dict

{0: [{'coor_init': (0.9583519177297942, 0.3168587380573808, 0.331039169536208),
   'coor_final_closest': (0.966044154398261,
    0.3283367829630452,
    0.3304881657415147),
   'dist': 0.013828218431957142}],
 1: [{'coor_init': (0.9943077422650554,
    0.7014125238969919,
    0.6961492118449879),
   'coor_final_closest': (0.9912961640487372,
    0.7866388388604441,
    0.8083328523902946),
   'dist': 0.14091757722726123}],
 2: [{'coor_init': (0.028584954670583, 0.2757325394423627, 0.683866194269555),
   'coor_final_closest': (0.02845629538145,
    0.200962518709988,
    0.8209024411843219),
   'dist': 0.15610735255591707}],
 3: [{'coor_init': (0.0731773730080812,
    0.7875552700544789,
    0.0790869644534405),
   'coor_final_closest': (0.0208759304880026,
    0.6999420232859563,
    0.2826071187348129),
   'dist': 0.22766636795477008}],
 4: [{'coor_init': (0.1817815183141048,
    0.1900679353891917,
    0.9741949097442956),
   'coor_final_closest': (0.3127535481871486,
    0.324512956

In [19]:
coor_final_reindex_dict_eliminated

{0: [{'coor_init': (0.9583519177297942, 0.3168587380573808, 0.331039169536208),
   'coor_final_closest': (0.966044154398261,
    0.3283367829630452,
    0.3304881657415147),
   'dist': 0.013828218431957142}],
 1: [{'coor_init': (0.9943077422650554,
    0.7014125238969919,
    0.6961492118449879),
   'coor_final_closest': (0.9912961640487372,
    0.7866388388604441,
    0.8083328523902946),
   'dist': 0.14091757722726123}],
 2: [{'coor_init': (0.028584954670583, 0.2757325394423627, 0.683866194269555),
   'coor_final_closest': (0.02845629538145,
    0.200962518709988,
    0.8209024411843219),
   'dist': 0.15610735255591707}],
 3: [{'coor_init': (0.0731773730080812,
    0.7875552700544789,
    0.0790869644534405),
   'coor_final_closest': (0.0208759304880026,
    0.6999420232859563,
    0.2826071187348129),
   'dist': 0.22766636795477008}],
 4: [{'coor_init': (0.1817815183141048,
    0.1900679353891917,
    0.9741949097442956),
   'coor_final_closest': (0.3127535481871486,
    0.324512956

In [20]:
coor_final_reindex_dict_eliminated_final

{0: [{'coor_init': (0.9583519177297942, 0.3168587380573808, 0.331039169536208),
   'coor_final_closest': (0.966044154398261,
    0.3283367829630452,
    0.3304881657415147),
   'dist': 0.013828218431957142}],
 1: [{'coor_init': (0.9943077422650554,
    0.7014125238969919,
    0.6961492118449879),
   'coor_final_closest': (0.9912961640487372,
    0.7866388388604441,
    0.8083328523902946),
   'dist': 0.14091757722726123}],
 2: [{'coor_init': (0.028584954670583, 0.2757325394423627, 0.683866194269555),
   'coor_final_closest': (0.02845629538145,
    0.200962518709988,
    0.8209024411843219),
   'dist': 0.15610735255591707}],
 3: [{'coor_init': (0.0731773730080812,
    0.7875552700544789,
    0.0790869644534405),
   'coor_final_closest': (0.0208759304880026,
    0.6999420232859563,
    0.2826071187348129),
   'dist': 0.22766636795477008}],
 4: [{'coor_init': (0.1817815183141048,
    0.1900679353891917,
    0.9741949097442956),
   'coor_final_closest': (0.3127535481871486,
    0.324512956

In [21]:
# duplicate_coor_final_w_data = get_duplicate_coor_final_closest_w_data(coor_final_reindex_dict)

In [22]:
# for key, value in duplicate_coor_final_w_data.items():
#     # Extract the list of coor_inits and dists
#     coor_inits_and_dists = value[0]['coor_inits and dists']
    
#     # # Find the coor_init with the biggest distance, and then delete it from coordinate
#     max_distance_coor_init = max(coor_inits_and_dists, key=lambda x: x['dist'])['coor_init']

#     # print("Duplicate coor_final_closest:", key)
#     # # for i in coor_inits_and_dists:
#     # #     print(i['coor_init'])
#     # print("coor_init with the biggest distance:", max_distance_coor_init)
#     # # print()

#     coor_final_reindex_dict_eliminated = coor_final_reindex_dict.copy()

#     # Find the corresponding entry and delete it
#     for idx, values in coor_final_reindex_dict_eliminated.items():
#         for entry in values:
#             if entry['coor_init'] == max_distance_coor_init and entry['coor_final_closest'] == key:
#                 entry['coor_final_closest'] = None
#                 entry['dist'] = None
#                 # del entry[0]['coor_final_closest']
#                 # del entry[0]['dist']
#                 # # entry['coor_init'] = max_distance_coor_init



In [23]:
coor_final_reindex_dict_eliminated

{0: [{'coor_init': (0.9583519177297942, 0.3168587380573808, 0.331039169536208),
   'coor_final_closest': (0.966044154398261,
    0.3283367829630452,
    0.3304881657415147),
   'dist': 0.013828218431957142}],
 1: [{'coor_init': (0.9943077422650554,
    0.7014125238969919,
    0.6961492118449879),
   'coor_final_closest': (0.9912961640487372,
    0.7866388388604441,
    0.8083328523902946),
   'dist': 0.14091757722726123}],
 2: [{'coor_init': (0.028584954670583, 0.2757325394423627, 0.683866194269555),
   'coor_final_closest': (0.02845629538145,
    0.200962518709988,
    0.8209024411843219),
   'dist': 0.15610735255591707}],
 3: [{'coor_init': (0.0731773730080812,
    0.7875552700544789,
    0.0790869644534405),
   'coor_final_closest': (0.0208759304880026,
    0.6999420232859563,
    0.2826071187348129),
   'dist': 0.22766636795477008}],
 4: [{'coor_init': (0.1817815183141048,
    0.1900679353891917,
    0.9741949097442956),
   'coor_final_closest': (0.3127535481871486,
    0.324512956

In [24]:
# coor_final_reindex_dict_eliminated[0][0]['coor_init']

In [25]:
# coor_final_reindex_dict_eliminated

In [26]:
# # coor_init_array = []
# coor_final_closest_eliminated_array = coor_final_closest.copy()

# for key, values in coor_final_reindex_dict_eliminated.items():
#     for entry in values:
#         if np.all(entry['coor_final_closest'] == coordinate):
#             coor_final_closest_eliminated_array.remove(entry['coor_final_closest'])


# # # coor_init_array = np.array(coor_init_array)
# # coor_final_closest_eliminated_array = np.array(coor_final_closest_eliminated_array)

In [27]:
# coor_final_closest_eliminated_array

In [28]:
# idx_empty = []

# for idx, values in coor_final_reindex_dict_eliminated.items():
#     for entry in values:
#         if entry['dist'] == None:
#             idx_empty.append(idx)
#             print(idx)

In [29]:
# coordinate

In [30]:
# coor_final_closest_array = []

# for i in idx_empty:
#     coor_final_closest_array.append(coordinate_init[i])

#     for idx, values in coor_final_reindex_dict_eliminated.items():
#         if i == idx:
#             coor_final_reindex_dict_eliminated[i][0]['coor_init'] = coordinate_init[i]

In [31]:
coordinate_final = []

for key, values in coor_final_reindex_dict_eliminated_final.items():
    for entry in values:
        coor_final_closest = entry['coor_final_closest']

        coordinate_final.append(coor_final_closest)

In [32]:
# duplicate_coor_final_w_data

In [33]:
# # Find duplicate rows and get their indices
# unique_rows, indices, counts = np.unique(coor_final_reindex, axis=0, return_index=True, return_counts=True)

# # Get the indices of duplicate rows
# duplicate_indices = indices[counts > 1]

# if len(duplicate_indices) > 0:
#     print("Duplicate rows found at indices:", duplicate_indices)
# else:
#     print("No duplicate rows found.")

In [34]:
# counts

In [35]:
# indices

In [36]:
are_equal = all(np.array_equal(coor_reindex, coor) for coor_reindex, coor in zip(coordinate_final, coordinate))

if are_equal:
    print("The arrays are the same.")
else:
    print("The arrays are different.")
    indices_diff = [i for i, (reindex, coord) in enumerate(zip(coordinate_final, coordinate)) if not np.array_equal(reindex, coord)]
    print("Indices where the arrays differ:", indices_diff)

The arrays are the same.


In [37]:
# df_coordinate = pd.DataFrame(coordinate)
# df_concat = pd.concat([df_coordinate, df_float_non_Li], ignore_index=True)

In [38]:
# for i in range(df_concat.shape[0]):
#     for j in range(df_concat.shape[1]):
#         temp = df_concat[j][i]
#         df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)

In [39]:
df_coordinate = pd.DataFrame(coordinate_final)
df_concat = pd.concat([df_coordinate, df_float_non_Li], ignore_index=True)

for i in range(df_concat.shape[0]):
    for j in range(df_concat.shape[1]):
        temp = df_concat[j][i]
        df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)

row_list = df_concat.to_string(index=False, header=False).split('\n')
row_list_space = ['  '.join(string.split()) for string in row_list] # 2 spaces of distance
row_list_w_beginning = ['  ' + row for row in row_list_space]       # 2 spaces in the beginning
absolute_correct_list = '\n'.join(row_list_w_beginning).splitlines()        

line_append_list = []
for idx_c, line in enumerate(absolute_correct_list):
    line_new_line = str(line) + '\n'
    line_append_list.append(line_new_line)

file_list = lines[:poscar_line_nr_start] + line_append_list

poscar_filename = file_path+"_reindexed"
destination_path = poscar_filename

with open(destination_path, "w") as poscar_positive_file:
    for item in file_list:
        poscar_positive_file.writelines(item)

/tmp/ipykernel_28504/3353428019.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat[j][i] = '{:.{width}f}'.format(float(temp), width=n_decimal)


In [40]:
mic_eucledian_distance((0.3447233333218262, 0.6487857534945276, 0.0476361236171942), (0.2153953041113269, 0.8290481479192707, 0.0136611114864024))

0.2244428021401018

In [41]:
mic_eucledian_distance((0.3447233333218262, 0.6487857534945276, 0.0476361236171942), (0.3035966699081504, 0.5380755899917785, 0.1598068290235508))

0.16288158243861353

In [42]:
mic_eucledian_distance((0.1686331230974980, 0.5481800698088471, 0.3216526124351740), (0.2153953041113269, 0.8290481479192707, 0.0136611114864024))

0.419442896627527

In [43]:
mic_eucledian_distance((0.1686331230974980, 0.5481800698088471, 0.3216526124351740), (0.3035966699081504, 0.5380755899917785, 0.1598068290235508))

0.21097705346365692